In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
# from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from transformers import pipeline

import tensorflow as tf
import torch


huggingface_api_key = os.getenv('HUGGING_FACE_API_KEY')

In [ ]:
pc=Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",dimension=1536,metric="cosine",spec=ServerlessSpec(cloud="aws",region="us-east-1")
)

In [ ]:
import requests

# Set up the headers and the API endpoint
headers = {
    "Authorization": f"Bearer {huggingface_api_key}"
}

api_url = "https://api-inference.huggingface.co/models/gpt2"

# Example data to be sent to the model
data = {
    "inputs": "Your input text here"
}

# Make a POST request to the Hugging Face API
response = requests.post(api_url, headers=headers, json=data)

# Output the response
print(response.json())

In [ ]:
import json
data=json.load(open("reviews.json"))
data['reviews']

In [ ]:
print(huggingface_api_key)

In [ ]:
# with open('reviews.json', 'r') as f:
#     data = json.load(f)

In [ ]:
import torch
print(torch.__version__)

import tensorflow as tf
print(tf.__version__)

In [ ]:
processed_data=[]
# client = OpenAI()
# Initialize the Hugging Face pipeline
embedding_pipeline = pipeline("feature-extraction", model="distilbert-base-uncased", use_auth_token=huggingface_api_key)

for review in data['reviews']:
    embedding = embedding_pipeline(review['review'])[0]

    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })